In [2]:
import re
import requests
import math
import random
from math import dist
from lxml import html
from collections import Counter
from string import punctuation as enPunc
from zhon.hanzi import punctuation as zhPunc

import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from bs4  import BeautifulSoup
from LAC import LAC
from opencc import OpenCC
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/lib/python3.7/site-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict
/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/opt/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:73: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import namedtuple, defaultdict, Iterable
INFO:gensim.summarization.textcleaner:'pattern' package not found; tag filters are not available for English


## 1. 資料處理與特徵萃取

### 資料處理：爬蟲、資料清洗、分詞、命名實體辨識

In [3]:
def multi_repl(input_str):
    trim = ['\n', '\r', '\t', '\xa0', ' '] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點 (無數字)
    trim_dict = {pat:'' for pat in trim}
    for k in trim_dict:
        input_str = input_str.replace(k, trim_dict[k])  ## 把stop words改為空白
    return input_str

def getFinalText(google_search_rs):
    import time
    html_text = []
    for idx, link in enumerate(google_search_rs):
        print(idx +1, link)
        if 'pdf' in link or 'PDF' in link or 'download' in link or 'Download' in link:
            html_text.append('無結果')
            continue
        try:
            t1 = time.time()
            print(time.asctime(time.localtime(t1)))
            resp = requests.get(link, headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}, timeout=10)
            soup = BeautifulSoup(resp.content, 'html.parser')              
            a_tag = soup.find_all('a')
            for a in a_tag:
                a.decompose() 
            html_text.append(soup.get_text()) 
        except Exception as e:
            print(e)
            html_text.append('無結果') 
                 
    
    # 繁轉簡
    cc = OpenCC('tw2sp')
    temp = [cc.convert(texts) for texts in html_text]

    # 只保留漢字、數字
    temp1 = [re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039])","", texts) for texts in temp]

    # 去除停用詞 (先去除還是等到分詞時再去除)
    final_text = [multi_repl(texts) for texts in temp1]
    
    # 紀錄資訊量少的文章 (文字少於400者)
    lessInfo = [i+1 for i in range(len(final_text)) if len(final_text[i]) < 400]
    
    return final_text, lessInfo

In [148]:
# 搜尋結果頁所有文章
google_search_rs = pd.read_excel('data/test.xlsx').url.to_list() 
print(len(google_search_rs))
final_text, lessInfo = getFinalText(google_search_rs)

31
1 https://zh.wikipedia.org/zh-tw/%E4%B8%AD%E5%9B%BD%E5%9B%BD%E9%98%B2%E7%A7%91%E6%8A%80%E4%BF%A1%E6%81%AF%E4%B8%AD%E5%BF%83
Mon Aug 30 14:50:31 2021
2 http://www.sohu.com/a/205770974_114731
Mon Aug 30 14:50:33 2021
3 http://scitech.people.com.cn/BIG5/n1/2017/1121/c1007-29660043.html
Mon Aug 30 14:50:33 2021
4 http://218.90.174.174:8080/opac/item.php?marc_no=644b4d484e30304d6d5a39446e2f7a5346426f494d673d3d
Mon Aug 30 14:50:33 2021
5 http://www.jnfhwl.com/mingxing/gukrlsrpn.html
Mon Aug 30 14:50:34 2021
6 https://book.douban.com/subject/26545176//
Mon Aug 30 14:50:34 2021
7 https://baike.baidu.com/item/%E8%B5%B5%E4%BC%8A%E5%90%9B/9157898
Mon Aug 30 14:50:35 2021
8 https://www.shimenmuyu.com/art/2019/12/25/art_23348_448376.html
Mon Aug 30 14:50:36 2021
HTTPSConnectionPool(host='www.shimenmuyu.com', port=443): Max retries exceeded with url: /art/2019/12/25/art_23348_448376.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x141e073d0>: Failed to establish

2021-08-30 14:50:41,477 - WARNING - Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


16 https://netl.istic.ac.cn/sso/tc46/details/9EEB494E5936427C918C19E0201E94D8
Mon Aug 30 14:50:41 2021
HTTPSConnectionPool(host='netl.istic.ac.cn', port=443): Max retries exceeded with url: /sso/tc46/details/9EEB494E5936427C918C19E0201E94D8 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
17 http://yantao.wanfangdata.com.cn/Home/GetTrendDetail?id=0dba5263-e87a-4fdc-92a3-a96cef1a1ea8
Mon Aug 30 14:50:41 2021
18 http://www.diobosi.com/contents/1353/7533.html
Mon Aug 30 14:50:42 2021
HTTPConnectionPool(host='www.diobosi.com', port=80): Read timed out. (read timeout=10)
19 http://vietyeah.com/contents/1345/9301.html
Mon Aug 30 14:50:52 2021
20 http://www.dsti.net/
Mon Aug 30 14:50:52 2021
21 http://pkunews.pku.edu.cn/xwzh/index1177.htm
Mon Aug 30 14:50:52 2021
22 http://222.88.118.79:82/KCMS/detail/detail.aspx?filename=JSSG201507015&dbcode=CJFD&dbname=CJFD2015
Mon Aug 30 14:

In [149]:
for i in range(len(final_text)):
    if len(final_text[i]) < 400:
        print(i+1)
        print(google_search_rs[i])
        print(final_text[i])
        print('\n')
lessInfo

4
http://218.90.174.174:8080/opac/item.php?marc_no=644b4d484e30304d6d5a39446e2f7a5346426f494d673d3d
无锡科技职业学院图书馆书目检索系统无锡科技职业学院图书馆书目检索系统中文借阅关系图相关资源把本书分享到二维码收藏此书的书架状态文献类型浏览次数17豆瓣简介全部细节信息此书刊没有复本此书刊可能正在订购中或者处理中显示全部馆藏信息借阅趋势同名作者的其他著作点击查看用户名密码验证码请输入下面显示的内容证件号条码号姓名手机号送书地收藏到无锡科技职业学院图书馆56120101919992021


5
http://www.jnfhwl.com/mingxing/gukrlsrpn.html
404404您要找的内容已被删除


8
https://www.shimenmuyu.com/art/2019/12/25/art_23348_448376.html
无结果


9
https://amp.cn.shops-com.ru/5684017/1/%E4%B8%AD%E5%9B%BD%E5%9B%BD%E9%98%B2%E7%A7%91%E6%8A%80%E4%BF%A1%E6%81%AF%E4%B8%AD%E5%BF%83.html
无结果


13
http://www.lis.ac.cn/CN/article/downloadArticleFile.do?attachType=PDF&id=21795
无结果


16
https://netl.istic.ac.cn/sso/tc46/details/9EEB494E5936427C918C19E0201E94D8
无结果


18
http://www.diobosi.com/contents/1353/7533.html
无结果


19
http://vietyeah.com/contents/1345/9301.html
男同全黄全肉动漫日本又粗又黄视频图片男孩子做哭另一个男孩子肉车固定在机器上强制高潮


20
http://www.dsti.net/
国防科技信息网中国最权威的国防科技信息的专业门户国际军事新闻国内军事新闻热点军事专题中国独家的军事图片军事视频权威国防科技信息门户选择类别国防防务新闻视点图片视

[4, 5, 8, 9, 13, 16, 18, 19, 20, 22, 24, 25, 26, 28, 30, 31]

In [78]:
def stopwords():
    # Load stop words
    f = open('/Users/csti-user/Downloads/NER/stopwords-master/baidu_stopwords.txt','r')
    k = f.readlines()
    f.close()
    stopwords = [lines.strip('\n') for lines in k]

    # 建構stopwords
    trim = ['\n', '\r', '\t', '\xa0'] + list(enPunc)+list(zhPunc) ## 網頁亂碼+英文標點+中文標點
    trim = trim + stopwords  # list of all stopwords
    return trim

# 命名實體辨識、計算詞頻
def word_segments(final_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    trim = stopwords()
    df_all = pd.DataFrame(columns=['words', 'ner', 'doc'])
    cnt = []
    for i in range(len(final_text)):
        ner_model = lac.run(final_text[i])
        for k in range(len(ner_model[0])):
            if ner_model[0][k] not in trim and ner_model[0][k] != '' and len(ner_model[0][k])>1 and ner_model[1][k] != '' and ner_model[1][k] != 'f' and ner_model[1][k] != 'nw' and ner_model[1][k] != 'm' and ner_model[1][k] != 'c' and ner_model[1][k] != 'u' and ner_model[1][k] != 'q' and ner_model[1][k] != 'p' and ner_model[1][k] != 's' and ner_model[1][k] != 'r' and ner_model[1][k] != 'xc' and ner_model[1][k] != 'w':
#                 if all(str.isdigit() for str in ner_model[0][k]) == False: #所有字都不是數字
                if any(str.isdigit() for str in ner_model[0][k]) == False: # 
#                         ner_model[0][k] = multi_repl(ner_model[0][k])
                    df_all = df_all.append({'words': ner_model[0][k], 'ner': ner_model[1][k], 'doc': i+1}, ignore_index=True)
        cnt.append(Counter(df_all[df_all.doc == i+1]['words']))
    return cnt

In [79]:
# 所有文章的分詞詞頻
cnt = word_segments(final_text)
cnt

[Counter({'中国国防科技信息中心': 11,
          '机构': 7,
          '概况': 1,
          '上级': 3,
          '方式': 1,
          '总部': 2,
          '地址': 1,
          '沿革': 1,
          '成立': 2,
          '时间': 1,
          '位于': 1,
          '国家级': 2,
          '信息': 6,
          '简介': 1,
          '年代': 2,
          '情报': 2,
          '咨询': 1,
          '研究': 7,
          '软科学': 1,
          '服务': 2,
          '信息技术': 1,
          '开发': 1,
          '成果': 1,
          '专利': 1,
          '管理': 3,
          '人才培养': 1,
          '一体': 1,
          '科技': 5,
          '中心': 3,
          '图书馆': 1,
          '馆藏': 1,
          '文献': 2,
          '数据': 2,
          '数据库': 1,
          '声像': 1,
          '中国人民解放军': 1,
          '规模': 1,
          '系统': 2,
          '资源': 2,
          '最为': 1,
          '数字化': 1,
          '加工': 1,
          '方向': 1,
          '军事': 1,
          '高新技术': 1,
          '武器': 2,
          '发展': 1,
          '军备': 1,
          '控制': 1,
          '装备': 1,
          '建设': 2,
      

In [50]:
for i, count in enumerate(cnt):
    print(i, count)

0 ['深圳', '振华数据', '信息技术维基百科自由的百科全书', '深圳', '振华数据', '信息技术维基百科自由的百科全书', '深圳', '振华数据', '信息技术有限公司', '成立', '位于', '技术', '公司', '公司', '深圳', '北京', '设有', '办公室', '全球', '国家', '地区', '设立', '数据处理', '中心', '宣称', '集成', '全球', '开源', '数据', '助力', '中华民族', '复兴', '首席', '行政官', '合作', '伙伴', '致力于', '军民', '融合', '公司', '持有', '企业', '控股', '子公司', '申请', '专利', '涉及', '新闻', '信息', '抓取', '数据', '视频', '社交', '媒体', '操控', '注册', '仿真', '社交', '媒体', '交互', '专利', '收集', '信息', '事件', '深圳', '工作', '美国', '学者', '经济学', '教授', '鲍尔丁', '揭露', '公司', '设立', '数据库', '中国', '军方', '收集', '世界', '万人', '数据', '以此', '证实', '中国', '搜集', '数据', '影响', '控制', '外国', '事务', '猜测', '企业', '服务', '对象', '包括', '企业', '收集', '信息', '涉及', '出生', '日期', '地址', '婚姻', '状况', '照片', '政治', '属性', '亲属', '社交', '帐户', '包括', '新闻', '犯罪', '记录', '企业', '不端', '收集', '信息', '包括', '澳大利亚人', '信息', '收集', '公开', '也有', '信息', '来自', '私密', '银行', '记录', '工作', '申请表', '心理', '状况', '记录', '另有', '数据', '来源于', '振华', '公司', '回应', '英国', '采访', '相关', '报导', '失实', '公司', '私营企业', '中国政府', '军方', '无关', '信息', '作家', '杰里米柯克', '振华', '收集', '公开', 

## 2. 搜尋結果排序

### 取得文章關鍵字

In [80]:
# 计算tf
def tf(word, count):
    return count[word] / sum(count.values())
# 计算count_list有多少个文件包含word
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)

# 计算idf
def idf(word, count_list):
    return math.log2(len(count_list) / (n_containing(word, count_list)))    #对数以2为底
# 计算tf-idf
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

In [81]:
def tfidfDF():
    tfidf_lst = []
    for i, count in enumerate(cnt):
        print("Top words in document %d"%(i + 1))
        scores = {word: tfidf(word, count, cnt) for word in count}
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)    #type=list
        for word, score in sorted_words[:20]:
            tfidf_lst.append((word, round(score,5), i+1))
            print("    Word: %s, TF-IDF: %s"%(word, round(score, 5)))
    tfidf_df = pd.DataFrame(tfidf_lst, columns = ['words', 'tfidf', 'doc'])
    return tfidf_df

In [82]:
# 取得各篇文章的關鍵字
tfidf_df = tfidfDF()
tfidf_df

Top words in document 1
    Word: 少将, TF-IDF: 0.38232
    Word: 中将, TF-IDF: 0.23895
    Word: 海军, TF-IDF: 0.089
    Word: 上将, TF-IDF: 0.07965
    Word: 空军, TF-IDF: 0.07965
    Word: 中国国防科技信息中心, TF-IDF: 0.07593
    Word: 副部长, TF-IDF: 0.06372
    Word: 原始, TF-IDF: 0.06357
    Word: 主任, TF-IDF: 0.05163
    Word: 部长, TF-IDF: 0.05086
    Word: 上级, TF-IDF: 0.04779
    Word: 军委, TF-IDF: 0.04779
    Word: 副主任, TF-IDF: 0.04571
    Word: 机构, TF-IDF: 0.04399
    Word: 书记, TF-IDF: 0.03814
    Word: 单位, TF-IDF: 0.03386
    Word: 研究, TF-IDF: 0.03364
    Word: 国家级, TF-IDF: 0.03186
    Word: 硕士学位, TF-IDF: 0.03186
    Word: 撤销, TF-IDF: 0.03186
Top words in document 2
    Word: 科技, TF-IDF: 0.13058
    Word: 情报, TF-IDF: 0.1061
    Word: 创新, TF-IDF: 0.10529
    Word: 行业, TF-IDF: 0.1034
    Word: 建设, TF-IDF: 0.08587
    Word: 年会, TF-IDF: 0.07908
    Word: 助力, TF-IDF: 0.05931
    Word: 搜狐, TF-IDF: 0.04954
    Word: 今日, TF-IDF: 0.04954
    Word: 中心, TF-IDF: 0.04885
    Word: 服务, TF-IDF: 0.04885
    Word: 阅读,

,words,tfidf,doc
0,少将,0.38232,1
1,中将,0.23895,1
2,海军,0.08900,1
3,上将,0.07965,1
4,空军,0.07965,1
...,...,...,...
332,打击,0.02382,29
333,业务,0.02382,29
334,培育,0.02382,29
335,教科文组织,0.02382,29


### 增量訓練：預訓練的詞向量中，可能沒有搜尋詞的詞彙，所以用增量訓練方式擴增詞向量詞彙

In [83]:
def word_segments(input_text):
    lac = LAC(mode='lac')  
    lac.load_customization('custom_dict.txt')  # 自行增加字典
    trim = stopwords()
    segments = []
    for i in range(len(input_text)):
        ner_model = lac.run(input_text[i])
        temp = []
        for k in range(len(ner_model[0])):
            if ner_model[0][k] not in trim and len(ner_model[0][k])>1:
                if ner_model[0][k] != '' and ner_model[1][k] != '' and ner_model[1][k] != 'f' and ner_model[1][k] != 'm' and ner_model[1][k] != 'c' and ner_model[1][k] != 'u' and ner_model[1][k] != 'q' and ner_model[1][k] != 'p' and ner_model[1][k] != 's' and ner_model[1][k] != 'r' and ner_model[1][k] != 'xc' and ner_model[1][k] != 'w': 
                    if all(str.isdigit() for str in ner_model[0][k]) == False:
                        if any(str.isdigit() for str in ner_model[0][k]) == False:
                            temp.append(ner_model[0][k])
        segments.append(temp)
    return segments
segments = word_segments(final_text)

In [99]:
# 自定義需要納入訓練的詞
import gensim
from gensim import utils
def my_rule(word, count, min_count):
    if word in ['真溱','中国国防科技信息中心']:
        return gensim.utils.RULE_KEEP
    else:
        return gensim.utils.RULE_DEFAULT

In [100]:
model_cont = Word2Vec.load('model/word2vec_test.model')
print(model_cont)
model_cont.build_vocab(segments, update=True, trim_rule=my_rule)
model_cont.train(segments, total_examples=model_cont.corpus_count, epochs=model_cont.epochs)
print(model_cont)

2021-08-30 14:06:13,109 - INFO - loading Word2Vec object from model/word2vec_test.model
2021-08-30 14:06:13,346 - INFO - loading wv recursively from model/word2vec_test.model.wv.* with mmap=None
2021-08-30 14:06:13,346 - INFO - loading vectors from model/word2vec_test.model.wv.vectors.npy with mmap=None
2021-08-30 14:06:13,666 - INFO - loading syn1neg from model/word2vec_test.model.syn1neg.npy with mmap=None
2021-08-30 14:06:14,008 - INFO - setting ignored attribute cum_table to None
2021-08-30 14:06:17,867 - INFO - Word2Vec lifecycle event {'fname': 'model/word2vec_test.model', 'datetime': '2021-08-30T14:06:17.867069', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.6-x86_64-i386-64bit', 'event': 'loaded'}
2021-08-30 14:06:17,977 - INFO - collecting all words and their counts
2021-08-30 14:06:17,978 - INFO - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-30 14:06:17,979 - INFO - collected 2353

Word2Vec(vocab=455349, vector_size=250, alpha=0.025)


2021-08-30 14:06:19,538 - INFO - Word2Vec lifecycle event {'msg': 'added 3 new unique words (0.1274968125796855%% of original 2353) and increased the count of 73 pre-existing words (3.102422439439014%% of original 2353)', 'datetime': '2021-08-30T14:06:19.538841', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.6-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-08-30 14:06:19,540 - INFO - deleting the raw counts dictionary of 2353 items
2021-08-30 14:06:19,541 - INFO - sample=0.001 downsamples 76 most-common words
2021-08-30 14:06:19,542 - INFO - Word2Vec lifecycle event {'msg': 'downsampling leaves estimated 495.8153779895335 word corpus (34.2%% of prior 1449)', 'datetime': '2021-08-30T14:06:19.542477', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.6-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2021-08-30 14:06:23,437 - INFO - estimated required memor

2021-08-30 14:06:25,171 - INFO - EPOCH - 16 : training on 5499 raw words (4116 effective words) took 0.0s, 158083 effective words/s
2021-08-30 14:06:25,177 - INFO - worker thread finished; awaiting finish of 2 more threads
2021-08-30 14:06:25,179 - INFO - worker thread finished; awaiting finish of 1 more threads
2021-08-30 14:06:25,200 - INFO - worker thread finished; awaiting finish of 0 more threads
2021-08-30 14:06:25,201 - INFO - EPOCH - 17 : training on 5499 raw words (4132 effective words) took 0.0s, 149764 effective words/s
2021-08-30 14:06:25,205 - INFO - worker thread finished; awaiting finish of 2 more threads
2021-08-30 14:06:25,206 - INFO - worker thread finished; awaiting finish of 1 more threads
2021-08-30 14:06:25,225 - INFO - worker thread finished; awaiting finish of 0 more threads
2021-08-30 14:06:25,226 - INFO - EPOCH - 18 : training on 5499 raw words (4136 effective words) took 0.0s, 180105 effective words/s
2021-08-30 14:06:25,232 - INFO - worker thread finished; a

2021-08-30 14:06:25,604 - INFO - worker thread finished; awaiting finish of 2 more threads
2021-08-30 14:06:25,605 - INFO - worker thread finished; awaiting finish of 1 more threads
2021-08-30 14:06:25,620 - INFO - worker thread finished; awaiting finish of 0 more threads
2021-08-30 14:06:25,621 - INFO - EPOCH - 37 : training on 5499 raw words (4105 effective words) took 0.0s, 243483 effective words/s
2021-08-30 14:06:25,625 - INFO - worker thread finished; awaiting finish of 2 more threads
2021-08-30 14:06:25,626 - INFO - worker thread finished; awaiting finish of 1 more threads
2021-08-30 14:06:25,644 - INFO - worker thread finished; awaiting finish of 0 more threads
2021-08-30 14:06:25,644 - INFO - EPOCH - 38 : training on 5499 raw words (4118 effective words) took 0.0s, 190481 effective words/s
2021-08-30 14:06:25,650 - INFO - worker thread finished; awaiting finish of 2 more threads
2021-08-30 14:06:25,651 - INFO - worker thread finished; awaiting finish of 1 more threads
2021-08-

Word2Vec(vocab=455352, vector_size=250, alpha=0.025)


In [101]:
# model_cont.save('model/test.model')
print(list(model_cont.wv.key_to_index.keys())[455349:])

['中国国防科技信息中心', '真溱', '赵伊君']


### 匯入增量訓練過的詞向量

In [ ]:
# 測試1: 加上word != "" --- 455359
# 測試2: 在stopwords加上空格 --- 455359
# 測試3: 照原本思頡寫的
# 測試4: 重複增量訓練是否會有影響
# c7f675ac22d5496fb8b45dfaf8ee0322: [中国国防科技信息中心总工真溱研究员] 爬蟲時間過長，找不到搜尋詞 「找不到真溱」
# 8bc531516c934f458b9b1dfd3b0dc38e: [振華數據] 思頡說找不到搜尋詞 

In [69]:
model = Word2Vec.load('model/word2vec_test.model') #匯入儲存過的模型，可再強化訓練
print(model) #原本455349
print(list(model.wv.key_to_index.keys())[455349:])

2021-08-27 16:00:44,032 - INFO - loading Word2Vec object from model/word2vec_test.model
2021-08-27 16:00:44,266 - INFO - loading wv recursively from model/word2vec_test.model.wv.* with mmap=None
2021-08-27 16:00:44,267 - INFO - loading vectors from model/word2vec_test.model.wv.vectors.npy with mmap=None
2021-08-27 16:00:44,755 - INFO - loading syn1neg from model/word2vec_test.model.syn1neg.npy with mmap=None
2021-08-27 16:00:45,116 - INFO - setting ignored attribute cum_table to None
2021-08-27 16:00:49,078 - INFO - Word2Vec lifecycle event {'fname': 'model/word2vec_test.model', 'datetime': '2021-08-27T16:00:49.078148', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]', 'platform': 'macOS-10.14.6-x86_64-i386-64bit', 'event': 'loaded'}


Word2Vec(vocab=455361, vector_size=250, alpha=0.025)
['振华数据', '包定', '加拿大人', '社群', '鲍丁', '信息技术有限公司', '鲍尔丁', '外交部长', '混合战', '李坤仪', '中国国防科技信息中心', '赵伊君']


### 計算文本向量

In [40]:
# 詞向量與tfidf值加權平均，儲存為dict(文章向量)
doc_vec_dict = {k+1:[] for k in range(len(google_search_rs))}
for k in range(len(google_search_rs)):
    document = tfidf_df[tfidf_df.doc==k+1]
    doc_vec = np.zeros(250)
    wds = 0
    print('Doc %s'%(k+1))
    for s in range(len(document)):
        word = document.iloc[s,0]
        try:
            test_idx = model.wv.key_to_index[word]
            test_vec = model.wv.vectors[test_idx]
            tfidf = document.iloc[s,1]
            wd_vec = test_vec*tfidf
            doc_vec+=wd_vec
            wds+=1
#             print(wd_vec)
#             print('\n')
        except:
            print(word, "not in Word2Vec model!")
#             print('\n')
    if wds == 0:
        doc_vec_dict[k+1].append(doc_vec/(wds+1))
    else:
        doc_vec_dict[k+1].append(doc_vec/wds)
    print(wds)
    print('\n')

Doc 1
杰里米柯克 not in Word2Vec model!
黎堡美国 not in Word2Vec model!
18


Doc 2
克里斯托弗巴尔德 not in Word2Vec model!
航天系统 not in Word2Vec model!
包定 not in Word2Vec model!
17


Doc 3
克里斯托弗巴尔德 not in Word2Vec model!
航天系统 not in Word2Vec model!
资讯库 not in Word2Vec model!
包定 not in Word2Vec model!
16


Doc 4
包定 not in Word2Vec model!
遭情 not in Word2Vec model!
搜台人 not in Word2Vec model!
研判人为 not in Word2Vec model!
台湾科技公司 not in Word2Vec model!
15


Doc 5
鲍尔丁 not in Word2Vec model!
19


Doc 6
信息公司 not in Word2Vec model!
澳大利亚人 not in Word2Vec model!
剑桥分析公司 not in Word2Vec model!
克莱夫汉密尔顿 not in Word2Vec model!
16


Doc 7
把抓 not in Word2Vec model!
19


Doc 8
鲍尔丁 not in Word2Vec model!
19


Doc 9
印度新闻网 not in Word2Vec model!
吉尔默太空技术公司 not in Word2Vec model!
18


Doc 10
鲍尔丁 not in Word2Vec model!
孔安怡 not in Word2Vec model!
18


Doc 11
2


Doc 12
鲍丁 not in Word2Vec model!
19


Doc 13
信息技术有限公司 not in Word2Vec model!
19


Doc 14
政要个资 not in Word2Vec model!
汇整而成 not in Word2Vec model!
端云共生 not in Word2Vec model

### 計算文本相似性

In [46]:
# 找出搜尋詞的詞向量
"""
若詞向量中有此搜尋詞，則取得該詞彙之向量；若詞向量中無此搜尋詞，則進行分詞，再尋找該詞彙之向量
"""
test_idx = model.wv.key_to_index["振华"] #先查詢期望轉換成向量的詞彙位於模型的哪個位置，取出索引值
print(test_idx)
test_vec = model.wv.vectors[test_idx] #將索引值匯入模型，取出向量
print(test_vec)

8160
[ 0.00798745 -2.1551857  -0.61584455 -3.6931763  -1.4092041  -1.2288783
  0.11510973 -0.66181874 -0.6708649  -0.15478179 -0.53028214 -0.13515152
 -0.5527403   0.05648008 -1.3949339  -0.4952693   0.8231724   1.3339607
 -0.8064559  -0.21112761 -2.0151944   0.66322374 -1.407095   -1.2881776
 -0.03953554  0.54852587 -0.52896774 -1.0860178  -0.4095437   1.3473788
 -0.68359673  0.2320633  -0.47879255  1.6037294   2.813384    0.89737844
 -1.0943587   1.180737    0.5648706  -1.2292893   1.5960019  -0.56076133
 -0.317844   -1.9434716  -0.77176034 -1.5181133   3.996077    1.1230366
 -1.6417606   1.3142635   0.14096606 -0.9578659   1.3206234  -1.3073927
 -1.0537032  -0.48314947  2.1371884   1.7946806  -1.6171302  -0.44901842
 -0.11706249 -2.4419744  -1.3608899   2.277385   -1.05477    -0.53184634
  0.16641891  0.43494758 -1.1113983  -0.75243723 -0.9261528  -0.64467764
 -0.52925605 -0.05874157 -1.6227537  -0.71689826  0.29549992  0.33002844
 -3.0269701  -0.8295227   4.4602175  -1.5036347  -1.

In [271]:
# 計算文件與搜尋詞的距離
dist_dict = {i+1:[] for i in range(len(google_search_rs))}
for i in range(len(google_search_rs)):
    if i+1 in lessInfo:
        doc_dist = 100
    else:
        doc_dist = dist(doc_vec_dict[i+1][0],test_vec)
    dist_dict[i+1].append(doc_dist)
    print("Doc %s: ; Distance: %s"%(i+1, doc_dist))

Doc 1: ; Distance: 5.798754862592087
Doc 2: ; Distance: 100
Doc 3: ; Distance: 5.762987535617493
Doc 4: ; Distance: 5.762987535617493
Doc 5: ; Distance: 5.872854819210625
Doc 6: ; Distance: 5.822505764691033
Doc 7: ; Distance: 100
Doc 8: ; Distance: 100
Doc 9: ; Distance: 5.850077127945982
Doc 10: ; Distance: 5.901708439702747
Doc 11: ; Distance: 5.901708439702747
Doc 12: ; Distance: 100
Doc 13: ; Distance: 100
Doc 14: ; Distance: 5.518469063982106
Doc 15: ; Distance: 5.890697009223181
Doc 16: ; Distance: 5.8659750934797525
Doc 17: ; Distance: 5.774732419643136
Doc 18: ; Distance: 5.756618059168101
Doc 19: ; Distance: 5.832604056162044
Doc 20: ; Distance: 100
Doc 21: ; Distance: 5.715802465327816
Doc 22: ; Distance: 5.812400992217325
Doc 23: ; Distance: 100
Doc 24: ; Distance: 5.928477061000932
Doc 25: ; Distance: 5.761582235248773
Doc 26: ; Distance: 5.833994562952736
Doc 27: ; Distance: 5.719413833813439
Doc 28: ; Distance: 5.796523783910438
Doc 29: ; Distance: 5.734237354403623
Doc 

In [276]:
# 重新排序，由小到大(由短到遠)
sort_docs = sorted(dist_dict.items(), key=lambda x: x[1], reverse=False)
sort_docs

[(14, [5.29469885997154]),
 (35, [5.4393368789877234]),
 (27, [5.46952737310393]),
 (43, [5.476926253678557]),
 (30, [5.505512706995479]),
 (21, [5.507700308316133]),
 (29, [5.510948757723107]),
 (25, [5.525002902603239]),
 (31, [5.532096189902341]),
 (48, [5.536678663177973]),
 (39, [5.543535859479768]),
 (17, [5.543546819151679]),
 (18, [5.54621740447064]),
 (3, [5.549352231622719]),
 (4, [5.549352231622719]),
 (46, [5.560558149702795]),
 (44, [5.573737955131922]),
 (49, [5.585717707311369]),
 (28, [5.587055557433395]),
 (1, [5.589499391195412]),
 (22, [5.594459329660234]),
 (6, [5.596521624072691]),
 (19, [5.6089236720198805]),
 (26, [5.614441748092235]),
 (45, [5.631007021901257]),
 (9, [5.632376269741387]),
 (16, [5.65460605092105]),
 (5, [5.664689501290491]),
 (10, [5.68109859132106]),
 (11, [5.68109859132106]),
 (15, [5.6818818290339985]),
 (34, [5.689021389679419]),
 (32, [5.6992019846527295]),
 (24, [5.708137908052859]),
 (42, [5.777523296921371]),
 (2, [100]),
 (7, [100]),
 (

## 測試不同主題、不同輪搜尋的結果

### Topic 1 - 振華數據

In [ ]:
## Topic 1: Search 1 - 振华数据 word2vec_wiki_cont.model when vocabs = 455,361
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

In [181]:
## Topic 1: Search 1 - 振华数据 word2vec_wiki_cont.model when vocabs = 455,361
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

Rank 1: http://data.eastmoney.com/notices/detail/000733/AN202104291488653404.html
Rank 2: https://www.rfi.fr/cn/%E4%BA%9E%E6%B4%B2/20200916-%E5%8D%B0%E5%BA%A6%E5%A4%96%E9%95%B7%E5%B0%B1%E6%B7%B1%E5%9C%B3%E6%8C%AF%E8%8F%AF%E4%BA%8B%E4%BB%B6%E8%88%87%E4%B8%AD%E5%9C%8B%E9%A7%90%E5%8D%B0%E5%A4%A7%E4%BD%BF%E4%BA%A4%E6%B6%89
Rank 3: https://www.upmedia.mg/news_info.php?SerialNo=96103
Rank 4: https://www.epochtimes.com/gb/20/9/15/n12404579.htm
Rank 5: https://www.bbc.com/zhongwen/simp/chinese-news-54160466
Rank 6: https://www.dw.com/zh/%E4%B8%93%E8%AE%BF%E5%BC%80%E6%BA%90%E6%95%B0%E6%8D%AE%E6%81%90%E6%88%90%E4%B8%BA%E4%B8%AD%E5%9B%BD%E6%89%A9%E5%A4%A7%E5%BD%B1%E5%93%8D%E5%8A%9B%E7%9A%84%E5%88%A9%E5%99%A8/a-54942061
Rank 7: http://www.sohu.com/a/418438028_162522
Rank 8: https://yimedia.com.tw/news/55999/
Rank 9: https://www.sohfrance.org/jingbaoshenzhenzhenhuashujuwaixie-suosoujiquanqiudarenwuyinsiquanbaoguang/
Rank 10: https://www.newzealandreview.com/nznews/the-list-of-new-zealanders-that-in

In [59]:
## Topic 1: Search 2 - 振华数据王雪峰 word2vec_wiki_cont.model when vocabs = 455,363
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

Rank 1: https://www.enanyang.my/node/352725
Rank 2: http://www.bast.net.cn/art/2019/12/25/art_23348_448376.html
Rank 3: https://www.shangyexinzhi.com/article/2025970.html
Rank 4: https://www.epochtimes.com/gb/20/9/15/n12404579.htm
Rank 5: https://newsin.us/%E6%8C%AF%E5%8D%8E%E6%95%B0%E6%8D%AE%E5%85%A8%E7%90%83%E8%8C%83%E5%9B%B4%E7%9B%91%E6%8E%A7%E7%BB%86%E8%8A%82%E6%9B%9D%E5%85%89/
Rank 6: https://www.fountmedia.io/article/75492
Rank 7: https://tw.news.yahoo.com/%E4%B8%AD%E5%9C%8B-%E6%8C%AF%E8%8F%AF%E6%95%B8%E6%93%9A-%E8%92%90%E5%85%A8%E7%90%83%E5%80%8B%E8%B3%87-%E9%A9%9A%E8%A6%8B%E9%A6%AC%E8%8B%B1%E4%B9%9D%E7%AD%892%E5%8D%83%E5%8F%B0%E7%81%A3%E5%90%8D%E4%BA%BA-040228555.html
Rank 8: https://tw.appledaily.com/politics/20200920/RA2DTEHKGNCNHHQFUEM7PSZX4M/
Rank 9: https://6do.news/article/3189712
Rank 10: https://vct.news/news/f31dfd48-4f1c-4b6b-a300-f1b74a3af316
Rank 11: https://technews.tw/2020/10/03/personal-details-of-millions-around-world-gathered-by-china-tech-company/
Rank 12: htt

In [217]:
## Topic 1: Search 3 - 真溱研究員 word2vec_wiki_cont.model when vocabs = 455,370 真溱研究員
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

Rank 1: https://cbs.cau.edu.cn/art/2018/9/14/art_31827_585914.html
Rank 2: http://pkunews.pku.edu.cn/xwzh/2016-11/11/content_295789.htm
Rank 3: https://www.president.gov.tw/Page/294/43690/%E4%BB%BB%E5%85%8D%E5%AE%98%E5%93%A1-
Rank 4: http://www.lsc.org.cn/contents/1353/7531.html
Rank 5: http://www.jnfhwl.com/mingxing/gukrlsrpn.html
Rank 6: http://stdaily.com/index/kejixinwen/2019-11/13/content_814842.shtml
Rank 7: http://www.itapress.cn/CN/news/news34.shtml
Rank 8: http://www.sohu.com/a/205770974_114731
Rank 9: http://scitech.people.com.cn/BIG5/n1/2017/1121/c1007-29660043.html
Rank 10: http://www.pmo.cas.cn/xwdt2019/kpdt2019/202005/t20200527_5599393.html
Rank 11: http://www.qbzz.org/news/default/98.aspx
Rank 12: https://www.ithome.com.tw/news/129131
Rank 13: http://yantao.wanfangdata.com.cn/Home/GetTrendDetail?id=0dba5263-e87a-4fdc-92a3-a96cef1a1ea8
Rank 14: http://tgw.jlu.edu.cn/info/1003/1005.htm
Rank 15: http://218.90.174.174:8080/opac/item.php?marc_no=3068326f5152753149775942703968

In [220]:
## Topic 1: Search 3 - 真溱研究員 word2vec_wiki_cont.model when vocabs = 455,370 真溱
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

Rank 1: http://www.qbzz.org/news/default/98.aspx
Rank 2: http://stdaily.com/index/kejixinwen/2019-11/13/content_814842.shtml
Rank 3: http://www.sohu.com/a/205770974_114731
Rank 4: http://pkunews.pku.edu.cn/xwzh/2016-11/11/content_295789.htm
Rank 5: http://www.jnfhwl.com/mingxing/gukrlsrpn.html
Rank 6: http://scitech.people.com.cn/BIG5/n1/2017/1121/c1007-29660043.html
Rank 7: http://www.lsc.org.cn/contents/1353/7531.html
Rank 8: http://www.itapress.cn/CN/news/news34.shtml
Rank 9: https://cbs.cau.edu.cn/art/2018/9/14/art_31827_585914.html
Rank 10: http://www.pmo.cas.cn/xwdt2019/kpdt2019/202005/t20200527_5599393.html
Rank 11: https://s.ixueshu.com/?q=%E7%9C%9F%E6%BA%B1&author=%E7%9C%9F%E6%BA%B1
Rank 12: http://yantao.wanfangdata.com.cn/Home/GetTrendDetail?id=0dba5263-e87a-4fdc-92a3-a96cef1a1ea8
Rank 13: https://book.douban.com/subject/26545176//
Rank 14: https://www.ithome.com.tw/news/129131
Rank 15: http://www.bookschina.com/8090295.htm
Rank 16: http://tgw.jlu.edu.cn/info/1003/1005.htm
R

### Topic 2 - 上海發布 公眾號

In [239]:
## Topic 2: Search 1 - 上海发布 公众号 word2vec_wiki_cont.model when vocabs = 455,378
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

Rank 1: http://154.210.151.18/%E4%B8%8A%E6%B5%B7%E5%8F%91%E5%B8%83%E5%85%AC%E4%BC%97%E5%8F%B7%E5%AE%98%E7%BD%91_1502_1554.html
Rank 2: https://mp.weixin.qq.com/
Rank 3: https://news.cnstock.com/choice/wxszb/8
Rank 4: https://www.shzgh.org/renda/node5902/node5911/node6583/u1ai6276386.html
Rank 5: https://finance.sina.com.cn/tech/2021-06-04/doc-ikqcfnaz9149029.shtml
Rank 6: http://www.cnr.cn/shanghai/tt/20210501/t20210501_525477471.shtml
Rank 7: https://baike.sogou.com/v49041278.htm
Rank 8: http://www.stla.com.cn/index.php?m=content&c=index&a=show&catid=12&id=375
Rank 9: http://www.shio.gov.cn/sh/xwb/n809/n814/n1061/u1ai18626.html
Rank 10: https://finance.sina.cn/2021-06-05/detail-ikqcfnaz9198725.d.html
Rank 11: https://www.autohome.com.cn/news/202104/1135583.html
Rank 12: http://m.sh.bendibao.com/traffic/240272.html
Rank 13: http://www.sohu.com/a/443376014_205643
Rank 14: https://wemp.app/accounts/c3afe338-2047-4e08-af73-8e045a6db610
Rank 15: https://finance.sina.com.cn/tech/2021-06-17/

In [97]:
## Topic 2: Search 1 - 上海发布公众号
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

Rank 1: http://154.210.151.18/%E4%B8%8A%E6%B5%B7%E5%8F%91%E5%B8%83%E5%85%AC%E4%BC%97%E5%8F%B7%E5%AE%98%E7%BD%91_1502_1554.html
Rank 2: https://finance.sina.com.cn/tech/2021-06-17/doc-ikqciyzk0167895.shtml
Rank 3: http://www.stla.com.cn/index.php?m=content&c=index&a=show&catid=12&id=375
Rank 4: http://finance.eastmoney.com/a/202106171964351375.html
Rank 5: http://www.cnr.cn/shanghai/tt/20210501/t20210501_525477471.shtml
Rank 6: https://zh.wikipedia.org/wiki/%E4%B8%8A%E6%B5%B7%E7%99%BC%E4%BD%88
Rank 7: https://finance.sina.com.cn/tech/2021-06-04/doc-ikqcfnaz9149029.shtml
Rank 8: https://www.shiguangjiuguan.com/bec3/page_38
Rank 9: https://www.weixinba.cc/thread-6636-1-1.html
Rank 10: https://news.cnstock.com/choice/wxszb/8
Rank 11: https://shanghai.chinatax.gov.cn/was5/web/search?&channelid=198173&searchword=%E5%85%A8%E5%B8%82%E9%80%9A%E5%8A%9E&extrasql=%28doctitle%3D%E5%85%A8%E5%B8%82%E9%80%9A%E5%8A%9E+or+doccontent%3D%E5%85%A8%E5%B8%82%E9%80%9A%E5%8A%9E+or+CHARACTERNUMBER%3D%E5%85%A8%E

In [720]:
## Topic 2: Search 2 - 政务新媒体
for i in range(len(google_search_rs)):
    print("Rank %s:"%(i+1),google_search_rs[sort_docs[i][0]-1])

http://www.gxzf.gov.cn/appnew/
https://www.guilin.gov.cn/hdjl/zwxmt/
https://www.dl.gov.cn/col/col2611/index.html
http://www.anyang.gov.cn/zwgk/fdzdgknr/qtzdgknr/zfwzhzwxmtjgqk/
http://www.gaomi.gov.cn/gmsxxgk/xxgk/zwwx/
http://www.moe.gov.cn/s78/A01/A01_ztzl/A01_zfpc/
http://czj.sh.gov.cn/zss/zmhd/wbwx/index.html
http://gkml.xiaogan.gov.cn/c/xnq/xnqzwxmtljrk.jhtml
http://www.sc.gov.cn/10462/c103108/zwgklist00.shtml
http://gxq.cq.gov.cn/zwxx_202/gxdt/wx/wap.html
https://app.www.gov.cn/govdata/zwxmtjz.html
http://www.dege.gov.cn/gzdgx/c101491/nav_list.shtml
http://zdpx.zju.edu.cn/zjupxkc_6404_419.html
http://www.yw.gov.cn/col/col1229187629/index.html
http://gdfs.customs.gov.cn/guangdong_sub/zwgk62/zwxmt84/index.html
http://www.chinays.gov.cn/col/col16801/index.html
http://www.dt.gov.cn/dtzww/zfwzyxmtjsygl/zfwzyxmtjsyxgl.shtml
http://zw.china.com.cn/2020-12/17/content_77022059.html
http://www.qzts.gov.cn/zwgk/ztzl/zwwjz/
https://baike.sogou.com/m/fullLemma?lid=179106072&g_ut=3
http://www